In [3]:
from __future__ import print_function
from intermine.webservice import Service
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord


In [5]:
def FetchTerminator(promoter_name:str): 
    seq = ''
    service = Service("https://yeastmine.yeastgenome.org/yeastmine/service")
    query = service.new_query("Gene")
    query.add_view(
        "secondaryIdentifier", "symbol", "length", "flankingRegions.direction",
        "flankingRegions.sequence.length", "flankingRegions.sequence.residues"
    )
    query.add_constraint("Gene", "LOOKUP", promoter_name, "S. cerevisiae", code="B")
    query.add_constraint("flankingRegions.direction", "=", "downstream", code="C")
    query.add_constraint("flankingRegions.distance", "=", "0.5kb", code="A")
    query.add_constraint("flankingRegions.includeGene", "=", "false", code="D")

    for row in query.rows():
        seq = row["flankingRegions.sequence.residues"]

    return seq

In [9]:
print(len(FetchTerminator('ADH1')))

500


In [16]:
%%time
def FetchMultipleTerminators(List_of_terminator_names):
    # #initializing 
    LIST_OF_BIOrecord_objects = []

    for i in range(0, len(List_of_terminator_names)):
        #fetching the seqs
        promoters_seq = SeqRecord(Seq(FetchTerminator(List_of_terminator_names[i])))
        promoters_seq.name = str(List_of_terminator_names[i])+ ' Terminator'
        promoters_seq.id = str(List_of_terminator_names[i])
        promoters_seq.description = 'Defined as being 0.5kb downstream of the stop codon and fetched from Intermines API'

        # Append to list 
        LIST_OF_BIOrecord_objects.append(promoters_seq)
    
    return LIST_OF_BIOrecord_objects

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.87 µs


In [17]:
Terminators_we_want_to_investigate = ['ADH1', 'CYC1']

In [18]:
terminators = FetchMultipleTerminators(Terminators_we_want_to_investigate)

In [22]:
terminators

[SeqRecord(seq=Seq('GCGAATTTCTTATGATTTATGATTTTTATTATTAAATAAGTTATAAAAAAAATA...ACG'), id='ADH1', name='ADH1 Terminator', description='Defined as being 0.5kb downstream of the stop codon and fetched from Intermines API', dbxrefs=[]),
 SeqRecord(seq=Seq('ACAGGCCCCTTTTCCTTTGTCGATATCATGTAATTAGTTATGTCACGCTTACAT...GAC'), id='CYC1', name='CYC1 Terminator', description='Defined as being 0.5kb downstream of the stop codon and fetched from Intermines API', dbxrefs=[])]

In [20]:
with open('Data/UP_DW_and_Marker/Terminators_ADH1_CYC1.fasta', 'w') as outfile:
    for i in range(0, len(terminators)):
        print(terminators[i].format("fasta"), file = outfile)